In [5]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime
from datetime import datetime as dt

from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools
import warnings
warnings.filterwarnings("ignore")

# Preprocess fundamental data

In [7]:
fund_df = pd.read_csv("sp500_fundamental_199601_202210.csv")

In [8]:
df_daily_price = pd.read_csv("sp500_price_19960101_20221021.csv")

In [9]:
len(df_daily_price.tic.unique())

1843

In [10]:
len(fund_df.tic.unique())

935

## 1.1 Use Trade date instead of quarterly report date

In [11]:
# use trade date instead of report quarterly date
times = list(fund_df['datadate']) # quarterly report date
for i in range(len(times)):
    quarter = (times[i] - int(times[i]/10000)*10000)
    if 1201 < quarter:
        times[i] = int(times[i]/10000 + 1)*10000 + 301
    if quarter <= 301:
        times[i] = int(times[i]/10000)*10000 + 301
    if 301 < quarter <= 601:
        times[i] = int(times[i]/10000)*10000 + 601
    if 601 < quarter <= 901:
        times[i] = int(times[i]/10000)*10000 + 901
    if 901 < quarter <= 1201:
        times[i] = int(times[i]/10000)*10000 + 1201
    time_tmp = times[i]
    year = int(time_tmp/10000)
    month = int(quarter/100)
    day = int(quarter - month*100)
#     if(time_tmp < 20171114):
#         while(dt.date(year, month, day).weekday() > 4):
#             time_tmp = time_tmp + 1
#             year = int(time_tmp/10000)
#             month = int((time_tmp - year*10000)/100)
#             day = int(time_tmp - year*10000 - month*100)
#         times[i] = time_tmp

In [12]:
times = pd.to_datetime(times,format='%Y%m%d')
fund_df['tradedate']=times

## 1.2 Calculate adjusted close price

In [13]:
fund_df['adj_close_q'] = fund_df.prccq/fund_df.adjex

### match tickers and gvkey for fundamental and price data

In [14]:
tic_to_gvkey = {}
df_daily_groups = list(df_daily_price.groupby('tic'))

In [15]:
for tic, df_ in df_daily_groups:
    tic_to_gvkey[tic] = df_.gvkey.iloc[0]

In [16]:
fund_df.shape

(77711, 654)

In [17]:
sum(np.isin(fund_df.tic, list(tic_to_gvkey.keys()))==False)

100

In [18]:
fund_df = fund_df[np.isin(fund_df.tic, list(tic_to_gvkey.keys()))]

In [19]:
fund_df.shape

(77611, 654)

In [20]:
sum(np.isin(fund_df.tic, list(tic_to_gvkey.keys()))==False)

0

In [21]:
len(fund_df.gvkey.unique())

930

In [22]:
fund_df['gvkey'] = [tic_to_gvkey[x] for x in fund_df['tic']]

In [23]:
len(fund_df.gvkey.unique())

930

In [24]:
fund_df['date'] = fund_df["tradedate"]
#fund_df.drop('tradedate', axis=1, inplace=True)

In [25]:
fund_df['date']=pd.to_datetime(fund_df['date'], format="%Y%m%d")
fund_df.drop_duplicates(["date","gvkey"], keep='last',inplace=True)

### Get next quarter's return

In [26]:
l_df = list(fund_df.groupby('gvkey'))
for tic,df in l_df:
    df.reset_index(inplace=True, drop=True)
    df.sort_values('date')
    # our goal is to predict next quarter's return
    df['y_return'] = np.log(df['adj_close_q'].shift(-1) / df['adj_close_q'])

In [30]:
fund_df = pd.concat([x[1] for x in l_df])

In [31]:
fund_df.shape

(77585, 656)

In [32]:
fund_df.head()

,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,tic,...,gsector,gsubind,sic,spcindcd,spcseccd,spcsrc,tradedate,adj_close_q,date,y_return
0,1045,19960331,1996,1.0,12,INDL,C,D,STD,AAL,...,20.0,20302010.0,4512.0,605.0,600.0,C,1996-06-01,44.7500,1996-06-01,0.016621
1,1045,19960630,1996,2.0,12,INDL,C,D,STD,AAL,...,20.0,20302010.0,4512.0,605.0,600.0,C,1996-09-01,45.5000,1996-09-01,-0.133531
2,1045,19960930,1996,3.0,12,INDL,C,D,STD,AAL,...,20.0,20302010.0,4512.0,605.0,600.0,C,1996-12-01,39.8125,1996-12-01,0.101428
3,1045,19961231,1996,4.0,12,INDL,C,D,STD,AAL,...,20.0,20302010.0,4512.0,605.0,600.0,C,1997-03-01,44.0625,1997-03-01,-0.065958
4,1045,19970331,1997,1.0,12,INDL,C,D,STD,AAL,...,20.0,20302010.0,4512.0,605.0,600.0,C,1997-06-01,41.2500,1997-06-01,0.114410


## 1.3 Calculate Financial Ratios

In [33]:
fund_df['pe'] = fund_df.prccq / fund_df.epspxq
fund_df['ps'] = fund_df.prccq / (fund_df.revtq/fund_df.cshoq)
fund_df['pb'] = fund_df.prccq / ((fund_df.atq-fund_df.ltq)/fund_df.cshoq)

In [34]:
items = [
    'date', # Date
    'gvkey',#gvkey unique identifier
    'tic', # Ticker
    'gsector',#gics sector 11
    'oiadpq', # Quarterly operating income
    'revtq', # Quartely revenue
    'niq', # Quartely net income
    'atq', # Total asset
    'teqq', # Shareholder's equity
    'epspiy', # EPS(Basic) incl. Extraordinary items
    'ceqq', # Common Equity
    'cshoq', # Common Shares Outstanding
    'dvpspq', # Dividends per share
    'actq', # Current assets
    'lctq', # Current liabilities
    'cheq', # Cash & Equivalent
    'rectq', # Recievalbles
    'cogsq', # Cost of  Goods Sold
    'invtq', # Inventories
    'apq',# Account payable
    'dlttq', # Long term debt
    'dlcq', # Debt in current liabilites
    'ltq', # Liabilities   
    'pe', #Price–earnings ratio
    'ps', #Price–sales ratio
    'pb', #Price-to-Book (P/B) Ratio
    'adj_close_q',#adjusted close price
    'y_return' #next quarter return
]

# Omit items that will not be used
fund_data = fund_df[items]


In [35]:
fund_data.head()

,date,gvkey,tic,gsector,oiadpq,revtq,niq,atq,teqq,epspiy,...,invtq,apq,dlttq,dlcq,ltq,pe,ps,pb,adj_close_q,y_return
0,1996-06-01,1045,AAL,20.0,401.0,4308.0,157.0,19481.0,3896.0,2.02,...,606.0,880.0,6720.0,294.0,15585.0,44.306931,1.599698,1.768866,44.7500,0.016621
1,1996-09-01,1045,AAL,20.0,586.0,4550.0,293.0,19662.0,5022.0,5.44,...,605.0,856.0,5469.0,273.0,14640.0,27.164179,1.820000,1.648945,45.5000,-0.133531
2,1996-12-01,1045,AAL,20.0,588.0,4562.0,282.0,20149.0,5293.0,8.53,...,623.0,912.0,5432.0,264.0,14856.0,26.021242,1.588311,1.368954,39.8125,0.101428
3,1997-03-01,1045,AAL,20.0,290.0,4333.0,284.0,20497.0,5668.0,11.63,...,633.0,1068.0,4542.0,554.0,14829.0,21.759259,1.850767,1.414851,44.0625,-0.065958
4,1997-06-01,1045,AAL,20.0,349.0,4426.0,152.0,20323.0,5824.0,1.67,...,611.0,990.0,4411.0,430.0,14499.0,49.401198,1.696227,1.289062,41.2500,0.114410


In [36]:
fund_data.shape

(77585, 28)

In [37]:
# Rename column names for the sake of readability
fund_data = fund_data.rename(columns={
    'oiadpq':'op_inc_q', # Quarterly operating income
    'revtq':'rev_q', # Quartely revenue
    'niq':'net_inc_q', # Quartely net income
    'atq':'tot_assets', # Assets
    'teqq':'sh_equity', # Shareholder's equity
    'epspiy':'eps_incl_ex', # EPS(Basic) incl. Extraordinary items
    'ceqq':'com_eq', # Common Equity
    'cshoq':'sh_outstanding', # Common Shares Outstanding
    'dvpspq':'div_per_sh', # Dividends per share
    'actq':'cur_assets', # Current assets
    'lctq':'cur_liabilities', # Current liabilities
    'cheq':'cash_eq', # Cash & Equivalent
    'rectq':'receivables', # Receivalbles
    'cogsq':'cogs_q', # Cost of  Goods Sold
    'invtq':'inventories', # Inventories
    'apq': 'payables',# Account payable
    'dlttq':'long_debt', # Long term debt
    'dlcq':'short_debt', # Debt in current liabilites
    'ltq':'tot_liabilities', # Liabilities   
})

In [38]:
fund_data.head()

,date,gvkey,tic,gsector,op_inc_q,rev_q,net_inc_q,tot_assets,sh_equity,eps_incl_ex,...,inventories,payables,long_debt,short_debt,tot_liabilities,pe,ps,pb,adj_close_q,y_return
0,1996-06-01,1045,AAL,20.0,401.0,4308.0,157.0,19481.0,3896.0,2.02,...,606.0,880.0,6720.0,294.0,15585.0,44.306931,1.599698,1.768866,44.7500,0.016621
1,1996-09-01,1045,AAL,20.0,586.0,4550.0,293.0,19662.0,5022.0,5.44,...,605.0,856.0,5469.0,273.0,14640.0,27.164179,1.820000,1.648945,45.5000,-0.133531
2,1996-12-01,1045,AAL,20.0,588.0,4562.0,282.0,20149.0,5293.0,8.53,...,623.0,912.0,5432.0,264.0,14856.0,26.021242,1.588311,1.368954,39.8125,0.101428
3,1997-03-01,1045,AAL,20.0,290.0,4333.0,284.0,20497.0,5668.0,11.63,...,633.0,1068.0,4542.0,554.0,14829.0,21.759259,1.850767,1.414851,44.0625,-0.065958
4,1997-06-01,1045,AAL,20.0,349.0,4426.0,152.0,20323.0,5824.0,1.67,...,611.0,990.0,4411.0,430.0,14499.0,49.401198,1.696227,1.289062,41.2500,0.114410


In [39]:
# set data type to series
date = fund_data['date'].to_frame('date').reset_index(drop=True)
tic = fund_data['tic'].to_frame('tic').reset_index(drop=True)
gvkey = fund_data['gvkey'].to_frame('gvkey').reset_index(drop=True)
adj_close_q = fund_data['adj_close_q'].to_frame('adj_close_q').reset_index(drop=True)
y_return = fund_data['y_return'].to_frame('y_return').reset_index(drop=True)
gsector = fund_data['gsector'].to_frame('gsector').reset_index(drop=True)
pe = fund_data['pe'].to_frame('pe').reset_index(drop=True)
ps = fund_data['ps'].to_frame('ps').reset_index(drop=True)
pb = fund_data['pb'].to_frame('pb').reset_index(drop=True)

In [40]:
# Calculate financial ratios

# Profitability ratios
# Operating Margin
OPM = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='OPM')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        OPM[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        OPM.iloc[i] = np.nan
    else:
        OPM.iloc[i] = np.sum(fund_data['op_inc_q'].iloc[i-3:i])/np.sum(fund_data['rev_q'].iloc[i-3:i])
OPM=pd.Series(OPM).to_frame().reset_index(drop=True)

# Net Profit Margin        
NPM = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='NPM')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        NPM[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        NPM.iloc[i] = np.nan
    else:
        NPM.iloc[i] = np.sum(fund_data['net_inc_q'].iloc[i-3:i])/np.sum(fund_data['rev_q'].iloc[i-3:i])
NPM=pd.Series(NPM).to_frame().reset_index(drop=True)

# Return On Assets
ROA = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='ROA')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        ROA[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        ROA.iloc[i] = np.nan
    else:
        ROA.iloc[i] = np.sum(fund_data['net_inc_q'].iloc[i-3:i])/fund_data['tot_assets'].iloc[i]
ROA=pd.Series(ROA).to_frame().reset_index(drop=True)

# Return on Equity
ROE = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='ROE')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        ROE[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        ROE.iloc[i] = np.nan
    else:
        ROE.iloc[i] = np.sum(fund_data['net_inc_q'].iloc[i-3:i])/fund_data['sh_equity'].iloc[i]        
ROE=pd.Series(ROE).to_frame().reset_index(drop=True)

# For calculating valuation ratios in the next subpart, calculate per share items in advance
# Earnings Per Share       
EPS = fund_data['eps_incl_ex'].to_frame('EPS').reset_index(drop=True)

# Book Per Share
BPS = (fund_data['com_eq']/fund_data['sh_outstanding']).to_frame('BPS').reset_index(drop=True) # Need to check units

#Dividend Per Share
DPS = fund_data['div_per_sh'].to_frame('DPS').reset_index(drop=True)

# Liquidity ratios
# Current ratio
cur_ratio = (fund_data['cur_assets']/fund_data['cur_liabilities']).to_frame('cur_ratio').reset_index(drop=True)

# Quick ratio
quick_ratio = ((fund_data['cash_eq'] + fund_data['receivables'] )/fund_data['cur_liabilities']).to_frame('quick_ratio').reset_index(drop=True)

# Cash ratio
cash_ratio = (fund_data['cash_eq']/fund_data['cur_liabilities']).to_frame('cash_ratio').reset_index(drop=True)


# Efficiency ratios
# Inventory turnover ratio
inv_turnover = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='inv_turnover')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        inv_turnover[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        inv_turnover.iloc[i] = np.nan
    else:
        inv_turnover.iloc[i] = np.sum(fund_data['cogs_q'].iloc[i-3:i])/fund_data['inventories'].iloc[i]
inv_turnover=pd.Series(inv_turnover).to_frame().reset_index(drop=True)

# Receivables turnover ratio       
acc_rec_turnover = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='acc_rec_turnover')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        acc_rec_turnover[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        acc_rec_turnover.iloc[i] = np.nan
    else:
        acc_rec_turnover.iloc[i] = np.sum(fund_data['rev_q'].iloc[i-3:i])/fund_data['receivables'].iloc[i]
acc_rec_turnover=pd.Series(acc_rec_turnover).to_frame().reset_index(drop=True)

# Payable turnover ratio
acc_pay_turnover = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='acc_pay_turnover')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        acc_pay_turnover[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        acc_pay_turnover.iloc[i] = np.nan
    else:
        acc_pay_turnover.iloc[i] = np.sum(fund_data['cogs_q'].iloc[i-3:i])/fund_data['payables'].iloc[i]
acc_pay_turnover=pd.Series(acc_pay_turnover).to_frame().reset_index(drop=True)

## Leverage financial ratios
# Debt ratio
debt_ratio = (fund_data['tot_liabilities']/fund_data['tot_assets']).to_frame('debt_ratio').reset_index(drop=True)

# Debt to Equity ratio
debt_to_equity = (fund_data['tot_liabilities']/fund_data['sh_equity']).to_frame('debt_to_equity').reset_index(drop=True)

In [41]:
# Create a dataframe that merges all the ratios
ratios = pd.concat([date,gvkey,tic,gsector,adj_close_q,y_return,OPM,NPM,ROA,ROE,EPS,BPS,DPS,
                    cur_ratio,quick_ratio,cash_ratio,inv_turnover,acc_rec_turnover,acc_pay_turnover,
                   debt_ratio,debt_to_equity,pe,ps,pb], axis=1).reset_index(drop=True)

In [44]:
ratios.shape

(77585, 24)

In [97]:
# Replace NAs infinite values with zero
final_ratios = ratios.copy()
final_ratios = final_ratios.fillna(0)
final_ratios = final_ratios.replace(np.inf,0)

In [46]:
final_ratios.to_csv('final_ratios_raw.csv')

In [56]:
final_ratios.shape

(77585, 24)

In [57]:
final_ratios.head()

,date,gvkey,tic,gsector,adj_close_q,y_return,OPM,NPM,ROA,ROE,...,quick_ratio,cash_ratio,inv_turnover,acc_rec_turnover,acc_pay_turnover,debt_ratio,debt_to_equity,pe,ps,pb
0,1996-06-01,1045,AAL,20.0,44.7500,0.016621,0.000000,0.000000,0.000000,0.000000,...,0.475086,0.179124,0.000000,0.000000,0.000000,0.800010,4.000257,44.306931,1.599698,1.768866
1,1996-09-01,1045,AAL,20.0,45.5000,-0.133531,0.000000,0.000000,0.000000,0.000000,...,0.497600,0.185348,0.000000,0.000000,0.000000,0.744583,2.915173,27.164179,1.820000,1.648945
2,1996-12-01,1045,AAL,20.0,39.8125,0.101428,0.000000,0.000000,0.000000,0.000000,...,0.557623,0.269708,0.000000,0.000000,0.000000,0.737307,2.806726,26.021242,1.588311,1.368954
3,1997-03-01,1045,AAL,20.0,44.0625,-0.065958,0.117362,0.054545,0.035713,0.129146,...,0.573662,0.325368,17.292259,9.710564,10.249064,0.723472,2.616267,21.759259,1.850767,1.414851
4,1997-06-01,1045,AAL,20.0,41.2500,0.114410,0.108888,0.063890,0.042267,0.147493,...,0.611247,0.329496,18.129296,9.005358,11.188889,0.713428,2.489526,49.401198,1.696227,1.289062


In [72]:
features_column_financial=[ 'OPM', 'NPM', 'ROA', 'ROE', 'EPS', 'BPS', 'DPS', 'cur_ratio',
       'quick_ratio', 'cash_ratio', 'inv_turnover', 'acc_rec_turnover',
       'acc_pay_turnover', 'debt_ratio', 'debt_to_equity', 'pe', 'ps', 'pb']

In [98]:
def handle_nan(df,features_column_financial):
    ##handle nan, inf
    df=df.drop(list(df[df.adj_close_q==0].index)).reset_index(drop=True)
    df['y_return'] = pd.to_numeric(df['y_return'], errors='coerce')
    for col in features_column_financial:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df['y_return'].replace([np.nan,np.inf,-np.inf], np.nan, inplace=True)
    df[features_column_financial].replace([np.nan, np.inf, -np.inf], np.nan, inplace=True)
    dropped_col = []
    for col in features_column_financial:
        if np.any(~np.isfinite(df[col])):
            df.drop(columns=[col], axis=1, inplace=True)
    df.dropna(axis=0, inplace=True)
    df=df.reset_index(drop=True)
    print("dropped_col: ",dropped_col)
    return df

In [99]:
final_ratios=handle_nan(final_ratios,features_column_financial)

dropped_col:  []


In [100]:
final_ratios.shape

(74017, 18)

In [101]:
#final_ratios[final_ratios.adj_close_q==0]

,date,gvkey,tic,gsector,adj_close_q,y_return,EPS,BPS,DPS,cur_ratio,quick_ratio,cash_ratio,acc_rec_turnover,debt_ratio,debt_to_equity,pe,ps,pb


In [104]:
final_ratios.date=final_ratios.date.apply(lambda x: x.strftime('%Y-%m-%d'))

In [105]:
final_ratios.shape

(74017, 18)

In [106]:
final_ratios.head()

,date,gvkey,tic,gsector,adj_close_q,y_return,EPS,BPS,DPS,cur_ratio,quick_ratio,cash_ratio,acc_rec_turnover,debt_ratio,debt_to_equity,pe,ps,pb
0,1996-06-01,1045,AAL,20.0,44.7500,0.016621,2.02,49.584416,0.0,0.717784,0.475086,0.179124,0.000000,0.800010,4.000257,44.306931,1.599698,1.768866
1,1996-09-01,1045,AAL,20.0,45.5000,-0.133531,5.44,55.186813,0.0,0.738051,0.497600,0.185348,0.000000,0.744583,2.915173,27.164179,1.820000,1.648945
2,1996-12-01,1045,AAL,20.0,39.8125,0.101428,8.53,58.164835,0.0,0.789316,0.557623,0.269708,0.000000,0.737307,2.806726,26.021242,1.588311,1.368954
3,1997-03-01,1045,AAL,20.0,44.0625,-0.065958,11.63,62.285714,0.0,0.803090,0.573662,0.325368,9.710564,0.723472,2.616267,21.759259,1.850767,1.414851
4,1997-06-01,1045,AAL,20.0,41.2500,0.114410,1.67,64.000000,0.0,0.851293,0.611247,0.329496,9.005358,0.713428,2.489526,49.401198,1.696227,1.289062


In [109]:
#final_ratios=final_ratios[final_ratios.date<'2022-12-01'].reset_index(drop=True)

In [110]:
#final_ratios.shape

(73917, 18)

In [111]:
final_ratios.to_csv('final_ratios.csv')

## 1.4 Separate by sector

In [112]:
for sec, df_ in list(final_ratios.groupby('gsector')):
    df_.to_excel(f"sector{int(sec)}.xlsx")